<a href="https://colab.research.google.com/github/washmore1/IS4100FinalProject/blob/main/PastAnalysis_WA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install nfl_data_py

In [ ]:
import pandas as pd
import numpy as np
import nfl_data_py as nfl
import seaborn as sns
import matplotlib.pyplot as plt

##**INTRO** - Will Ashmore
The goal of my NFL analysis with this project is to better familiarize myself with these 8 teams, gaining a wide-ranging scope of each franchise's place in their respective division, their conference, and in the league as a whole. These teams that I have selected have had and will have varying levels of success on the football field, and I hope to get a deeper look into some of the underlying statistics that can be part of the explanation as to why one franchise tends to be more successful than another.

Extending off of this, I am interested in looking at trends of some basic statistics, like passing yardage vs. rushing yardage, and defensive passing yards allowed vs. rushing yards allowed. This allows me to get my feet wet with some easier, more straightforward stats, while getting a better look at what a team is good at and what they struggle at. In addition, this will give a glimpse at the varying styles of play that each team in my selections deploy. Teams like the San Francisco 49ers and the Miami Dolphins have built strong reputations for running creative offenses, with unique formations and play concepts that are different from the traditional, pro-style offenses that the New Orleans Saints and Tampa Bay Buccaneers have utilized in recent memory.

##**Team Analysis (2022)**

## 2022 Season Results

In [ ]:
df = nfl.import_schedules([2022])
def calculate_win_loss(team, df):

    team_df = df[(df['home_team'] == team) | (df['away_team'] == team)]

    wins = 0
    losses = 0

    for _, row in team_df.iterrows():
        if (row['home_team'] == team and row['home_score'] > row['away_score']) or \
           (row['away_team'] == team and row['away_score'] > row['home_score']):
            wins += 1
        else:
            losses += 1

    return wins, losses


# List of teams
teams = ['MIA', 'PIT', 'HOU', 'DEN', 'WAS', 'CHI', 'TB', 'SF']

# Calculate and display the win-loss record for each team
for team in teams:
    wins, losses = calculate_win_loss(team, df)
    print(f"\n{team} 2022 Record: {wins} Wins, {losses} Losses")


MIA 2022 Record: 9 Wins, 9 Losses

PIT 2022 Record: 9 Wins, 8 Losses

HOU 2022 Record: 3 Wins, 14 Losses

DEN 2022 Record: 5 Wins, 12 Losses

WAS 2022 Record: 8 Wins, 9 Losses

CHI 2022 Record: 3 Wins, 14 Losses

TB 2022 Record: 8 Wins, 10 Losses

SF 2022 Record: 15 Wins, 5 Losses


In [ ]:
teams_to_check = ['MIA', 'PIT', 'HOU', 'DEN', 'WSH', 'CHI', 'TB', 'SF']

pbp_data_2022 = nfl.import_pbp_data([2022])
postseason_games = pbp_data_2022[pbp_data_2022['season_type'] == 'POST']

for team in teams_to_check:
    if (team in postseason_games['home_team'].values) or (team in postseason_games['away_team'].values):
        print(f"{team} Reached the NFL postseason.")
    else:
        print(f"{team} did not qualify for the NFL postseason.")

2022 done.
Downcasting floats.
MIA Reached the NFL postseason.
PIT did not qualify for the NFL postseason.
HOU did not qualify for the NFL postseason.
DEN did not qualify for the NFL postseason.
WSH did not qualify for the NFL postseason.
CHI did not qualify for the NFL postseason.
TB Reached the NFL postseason.
SF Reached the NFL postseason.


## 2022 Starting QB's

In [ ]:
depth_charts_2022 = nfl.import_depth_charts([2022])

my_teams = ['MIA', 'PIT', 'HOU', 'DEN', 'WAS', 'CHI', 'TB', 'SF']

starting_qbs = depth_charts_2022[
    (depth_charts_2022['position'] == 'QB') &
    (depth_charts_2022['depth_team'] == '1') &
    (depth_charts_2022['club_code'].isin(my_teams) &
    (depth_charts_2022['week'] == 15.0))

]
print(starting_qbs[['club_code', 'full_name', 'position', 'depth_team']])


      club_code        full_name position depth_team
3248        CHI    Justin Fields       QB          1
8324        DEN   Russell Wilson       QB          1
17350       MIA   Tua Tagovailoa       QB          1
27042       PIT    Kenny Pickett       QB          1
29333        SF  Jimmy Garoppolo       QB          1
31886        TB        Tom Brady       QB          1
33114       WAS  Taylor Heinicke       QB          1
37524       HOU      Davis Mills       QB          1


In [ ]:
# np.argmax()
qb_stats22 = nfl.import_seasonal_pfr('pass',[2022])
my_qbs = ['Tua Tagovailoa', 'Kenny Pickett', 'Davis Mills', 'Russell Wilson',
          'Taylor Heinicke', 'Justin Fields', 'Tom Brady', 'Jimmy Garoppolo']
print_qbs = qb_stats22[
    (qb_stats22['player'].isin(my_qbs))
]

print(print_qbs[['player', 'team', 'bad_throw_pct', 'on_tgt_pct']])

              player team  bad_throw_pct  on_tgt_pct
437        Tom Brady   TB           16.4        77.1
446   Tua Tagovailoa  MIA           15.8        76.7
447   Russell Wilson  DEN           15.6        73.3
450      Davis Mills  HOU           23.2        70.6
457  Jimmy Garoppolo   SF           12.7        74.5
458    Kenny Pickett  PIT           18.2        72.1
460    Justin Fields  CHI           19.3        71.1
465  Taylor Heinicke  WAS           18.1        72.8


## 2022 Passing Stats

In [ ]:
pbp_data_2022 = nfl.import_pbp_data([2022])

my_teams = ['MIA', 'PIT', 'HOU', 'DEN', 'WAS', 'CHI', 'TB', 'SF']

passing_plays = pbp_data_2022[pbp_data_2022['play_type'] == 'pass']

passing_plays_filtered = passing_plays[passing_plays['posteam'].isin(my_teams)]

passing_yards_per_game = passing_plays_filtered.groupby(['posteam', 'game_id'])['passing_yards'].sum().reset_index()

games_per_team = passing_yards_per_game.groupby('posteam')['game_id'].nunique().reset_index()
games_per_team.columns = ['team', 'total_games']

total_passing_yards_per_team = passing_yards_per_game.groupby('posteam')['passing_yards'].sum().reset_index()
total_passing_yards_per_team.columns = ['team', 'total_passing_yards']

team_stats = pd.merge(total_passing_yards_per_team, games_per_team, on='team')

team_stats['avg_passing_yards_per_game'] = team_stats['total_passing_yards'] / team_stats['total_games']

team_stats = team_stats.sort_values(by='avg_passing_yards_per_game', ascending=False)

print(team_stats[['team', 'total_passing_yards', 'total_games', 'avg_passing_yards_per_game']])

2022 done.
Downcasting floats.
  team  total_passing_yards  total_games  avg_passing_yards_per_game
6   TB               5097.0           18                  283.166667
3  MIA               4985.0           18                  276.944444
1  DEN               4007.0           17                  235.705882
5   SF               4692.0           20                  234.600000
7  WAS               3783.0           17                  222.529412
4  PIT               3661.0           17                  215.352941
2  HOU               3642.0           17                  214.235294
0  CHI               2598.0           17                  152.823529


## 2022 Rushing Stats

In [ ]:
pbp_data_2022 = nfl.import_pbp_data([2022])

my_teams = ['MIA', 'PIT', 'HOU', 'DEN', 'WAS', 'CHI', 'TB', 'SF']

rushing_plays = pbp_data_2022[pbp_data_2022['play_type'] == 'run']

rushing_plays_filtered = rushing_plays[rushing_plays['posteam'].isin(my_teams)]

rushing_yards_per_game = rushing_plays_filtered.groupby(['posteam', 'game_id'])['rushing_yards'].sum().reset_index()

games_per_team = passing_yards_per_game.groupby('posteam')['game_id'].nunique().reset_index()
games_per_team.columns = ['team', 'total_games']

total_rushing_yards_per_team = rushing_yards_per_game.groupby('posteam')['rushing_yards'].sum().reset_index()
total_rushing_yards_per_team.columns = ['team', 'total_rushing_yards']

team_stats = pd.merge(total_rushing_yards_per_team, games_per_team, on='team')

team_stats['avg_rushing_yards_per_game'] = team_stats['total_rushing_yards'] / team_stats['total_games']

team_stats = team_stats.sort_values(by='avg_rushing_yards_per_game', ascending=False)

print(team_stats[['team', 'total_rushing_yards', 'total_games', 'avg_rushing_yards_per_game']])

2022 done.
Downcasting floats.
  team  total_rushing_yards  total_games  avg_rushing_yards_per_game
0  CHI               3024.0           17                  177.882353
5   SF               2754.0           20                  137.700000
7  WAS               2154.0           17                  126.705882
4  PIT               2085.0           17                  122.647059
1  DEN               1944.0           17                  114.352941
3  MIA               1735.0           18                   96.388889
2  HOU               1485.0           17                   87.352941
6   TB               1373.0           18                   76.277778


## 2022 Passing Defense

In [ ]:
pbp_data_2022 = nfl.import_pbp_data([2022])

my_teams = ['MIA', 'PIT', 'HOU', 'DEN', 'WAS', 'CHI', 'TB', 'SF']

dpassing_plays = pbp_data_2022[pbp_data_2022['play_type'] == 'pass']

dpassing_plays_filtered = dpassing_plays[dpassing_plays['defteam'].isin(my_teams)]

dpassing_yards_per_game = dpassing_plays_filtered.groupby(['defteam', 'game_id'])['passing_yards'].sum().reset_index()

games_per_team = dpassing_yards_per_game.groupby('defteam')['game_id'].nunique().reset_index()
games_per_team.columns = ['team', 'total_games']

total_dpassing_yards_per_team = dpassing_yards_per_game.groupby('defteam')['passing_yards'].sum().reset_index()
total_dpassing_yards_per_team.columns = ['team', 'total_dpassing_yards']

team_stats = pd.merge(total_dpassing_yards_per_team, games_per_team, on='team')

team_stats['avg_dpassing_yards_per_game'] = team_stats['total_dpassing_yards'] / team_stats['total_games']

team_stats = team_stats.sort_values(by='avg_dpassing_yards_per_game', ascending=False)

print(team_stats[['team', 'total_dpassing_yards', 'total_games', 'avg_dpassing_yards_per_game']])

2022 done.
Downcasting floats.
  team  total_dpassing_yards  total_games  avg_dpassing_yards_per_game
3  MIA                4634.0           18                   257.444444
4  PIT                4066.0           17                   239.176471
5   SF                4676.0           20                   233.800000
6   TB                4097.0           18                   227.611111
0  CHI                3840.0           17                   225.882353
1  DEN                3819.0           17                   224.647059
2  HOU                3775.0           17                   222.058824
7  WAS                3531.0           17                   207.705882


## 2022 Rushing Defense

In [ ]:
pbp_data_2022 = nfl.import_pbp_data([2022])

my_teams = ['MIA', 'PIT', 'HOU', 'DEN', 'WAS', 'CHI', 'TB', 'SF']

drush_plays = pbp_data_2022[pbp_data_2022['play_type'] == 'run']

drush_plays_filtered = drush_plays[drush_plays['defteam'].isin(my_teams)]

drush_yards_per_game = drush_plays_filtered.groupby(['defteam', 'game_id'])['rushing_yards'].sum().reset_index()

games_per_team = drush_yards_per_game.groupby('defteam')['game_id'].nunique().reset_index()
games_per_team.columns = ['team', 'total_games']

total_drush_yards_per_team = drush_yards_per_game.groupby('defteam')['rushing_yards'].sum().reset_index()
total_drush_yards_per_team.columns = ['team', 'total_drush_yards']

team_stats = pd.merge(total_drush_yards_per_team, games_per_team, on='team')

team_stats['avg_drush_yards_per_game'] = team_stats['total_drush_yards'] / team_stats['total_games']

team_stats = team_stats.sort_values(by='avg_drush_yards_per_game', ascending=False)

print(team_stats[['team', 'total_drush_yards', 'total_games', 'avg_drush_yards_per_game']])

2022 done.
Downcasting floats.
  team  total_drush_yards  total_games  avg_drush_yards_per_game
2  HOU             2908.0           17                171.058824
0  CHI             2696.0           17                158.588235
6   TB             2192.0           18                121.777778
7  WAS             1940.0           17                114.117647
1  DEN             1881.0           17                110.647059
4  PIT             1860.0           17                109.411765
3  MIA             1888.0           18                104.888889
5   SF             1663.0           20                 83.150000


##**Team Analysis (2023)**

## 2023 Season Results

In [ ]:
df = nfl.import_schedules([2023])
def calculate_win_loss(team, df):

    team_df = df[(df['home_team'] == team) | (df['away_team'] == team)]

    wins = 0
    losses = 0

    for _, row in team_df.iterrows():
        if (row['home_team'] == team and row['home_score'] > row['away_score']) or \
           (row['away_team'] == team and row['away_score'] > row['home_score']):
            wins += 1
        else:
            losses += 1

    return wins, losses


# List of teams
teams = ['MIA', 'PIT', 'HOU', 'DEN', 'WAS', 'CHI', 'TB', 'SF']

# Calculate and display the win-loss record for each team
for team in teams:
    wins, losses = calculate_win_loss(team, df)
    print(f"\n{team} 2023 Record: {wins} Wins, {losses} Losses")


MIA 2023 Record: 11 Wins, 7 Losses

PIT 2023 Record: 10 Wins, 8 Losses

HOU 2023 Record: 11 Wins, 8 Losses

DEN 2023 Record: 8 Wins, 9 Losses

WAS 2023 Record: 4 Wins, 13 Losses

CHI 2023 Record: 7 Wins, 10 Losses

TB 2023 Record: 10 Wins, 9 Losses

SF 2023 Record: 14 Wins, 6 Losses


In [ ]:
teams_to_check = ['MIA', 'PIT', 'HOU', 'DEN', 'WSH', 'CHI', 'TB', 'SF']

pbp_data_2023 = nfl.import_pbp_data([2023])
postseason_games = pbp_data_2023[pbp_data_2023['season_type'] == 'POST']

for team in teams_to_check:
    if (team in postseason_games['home_team'].values) or (team in postseason_games['away_team'].values):
        print(f"{team} Reached the NFL postseason.")
    else:
        print(f"{team} did not qualify for the NFL postseason.")

2023 done.
Downcasting floats.
MIA Reached the NFL postseason.
PIT Reached the NFL postseason.
HOU Reached the NFL postseason.
DEN did not qualify for the NFL postseason.
WSH did not qualify for the NFL postseason.
CHI did not qualify for the NFL postseason.
TB Reached the NFL postseason.
SF Reached the NFL postseason.


## 2023 Starting QB's

In [ ]:
depth_charts_2023 = nfl.import_depth_charts([2023])

my_teams = ['MIA', 'PIT', 'HOU', 'DEN', 'WAS', 'CHI', 'TB', 'SF']

starting_qbs = depth_charts_2023[
    (depth_charts_2023['position'] == 'QB') &
    (depth_charts_2023['depth_team'] == '1') &
    (depth_charts_2023['club_code'].isin(my_teams) &
    (depth_charts_2023['week'] == 15.0))
]
print(starting_qbs[['club_code', 'full_name', 'position', 'depth_team']])

      club_code       full_name position depth_team
2494        CHI   Justin Fields       QB          1
7854        DEN  Russell Wilson       QB          1
17497       MIA  Tua Tagovailoa       QB          1
26629       PIT   Kenny Pickett       QB          1
28310        SF     Brock Purdy       QB          1
31399        TB  Baker Mayfield       QB          1
32703       WAS      Sam Howell       QB          1
36919       HOU     C.J. Stroud       QB          1


In [ ]:
qb_stats23 = nfl.import_seasonal_pfr('pass',[2023])
my_qbs = ['Tua Tagovailoa', 'Kenny Pickett', 'C.J. Stroud', 'Russell Wilson',
          'Sam Howell', 'Justin Fields', 'Baker Mayfield', 'Brock Purdy']
print_qbs = qb_stats23[
    (qb_stats23['player'].isin(my_qbs))
]

print(print_qbs[['player', 'team', 'bad_throw_pct', 'on_tgt_pct']])

             player team  bad_throw_pct  on_tgt_pct
541  Tua Tagovailoa  MIA           14.3        79.0
545     Brock Purdy   SF           16.3        75.5
548     C.J. Stroud  HOU           19.9        74.8
549  Baker Mayfield   TB           14.8        74.9
552      Sam Howell  WAS           15.2        73.7
559  Russell Wilson  DEN           13.2        77.9
562   Justin Fields  CHI           16.8        73.0
569   Kenny Pickett  PIT           18.4        71.6


## 2023 Passing Stats

In [ ]:
pbp_data_2023 = nfl.import_pbp_data([2023])

my_teams = ['MIA', 'PIT', 'HOU', 'DEN', 'WAS', 'CHI', 'TB', 'SF']

passing_plays = pbp_data_2023[pbp_data_2023['play_type'] == 'pass']

passing_plays_filtered = passing_plays[passing_plays['posteam'].isin(my_teams)]

passing_yards_per_game = passing_plays_filtered.groupby(['posteam', 'game_id'])['passing_yards'].sum().reset_index()

games_per_team = passing_yards_per_game.groupby('posteam')['game_id'].nunique().reset_index()
games_per_team.columns = ['team', 'total_games']

total_passing_yards_per_team = passing_yards_per_game.groupby('posteam')['passing_yards'].sum().reset_index()
total_passing_yards_per_team.columns = ['team', 'total_passing_yards']

team_stats = pd.merge(total_passing_yards_per_team, games_per_team, on='team')

team_stats['avg_passing_yards_per_game'] = team_stats['total_passing_yards'] / team_stats['total_games']

team_stats = team_stats.sort_values(by='avg_passing_yards_per_game', ascending=False)

print(team_stats[['team', 'total_passing_yards', 'total_games', 'avg_passing_yards_per_game']])

2023 done.
Downcasting floats.
  team  total_passing_yards  total_games  avg_passing_yards_per_game
3  MIA               4897.0           18                  272.055556
5   SF               5372.0           20                  268.600000
2  HOU               5033.0           19                  264.894737
6   TB               4730.0           19                  248.947368
7  WAS               4174.0           17                  245.529412
1  DEN               3566.0           17                  209.764706
4  PIT               3650.0           18                  202.777778
0  CHI               3421.0           17                  201.235294


## 2023 Rushing Stats

In [ ]:
pbp_data_2023 = nfl.import_pbp_data([2023])

my_teams = ['MIA', 'PIT', 'HOU', 'DEN', 'WAS', 'CHI', 'TB', 'SF']

rushing_plays = pbp_data_2023[pbp_data_2023['play_type'] == 'run']

rushing_plays_filtered = rushing_plays[rushing_plays['posteam'].isin(my_teams)]

rushing_yards_per_game = rushing_plays_filtered.groupby(['posteam', 'game_id'])['rushing_yards'].sum().reset_index()

games_per_team = passing_yards_per_game.groupby('posteam')['game_id'].nunique().reset_index()
games_per_team.columns = ['team', 'total_games']

total_rushing_yards_per_team = rushing_yards_per_game.groupby('posteam')['rushing_yards'].sum().reset_index()
total_rushing_yards_per_team.columns = ['team', 'total_rushing_yards']

team_stats = pd.merge(total_rushing_yards_per_team, games_per_team, on='team')

team_stats['avg_rushing_yards_per_game'] = team_stats['total_rushing_yards'] / team_stats['total_games']

team_stats = team_stats.sort_values(by='avg_rushing_yards_per_game', ascending=False)

print(team_stats[['team', 'total_rushing_yards', 'total_games', 'avg_rushing_yards_per_game']])

2023 done.
Downcasting floats.
  team  total_rushing_yards  total_games  avg_rushing_yards_per_game
0  CHI               2418.0           17                  142.235294
5   SF               2795.0           20                  139.750000
3  MIA               2403.0           18                  133.500000
4  PIT               2143.0           18                  119.055556
1  DEN               1831.0           17                  107.705882
7  WAS               1596.0           17                   93.882353
2  HOU               1760.0           19                   92.631579
6   TB               1738.0           19                   91.473684


## 2023 Passing Defense

In [ ]:
pbp_data_2023 = nfl.import_pbp_data([2023])

my_teams = ['MIA', 'PIT', 'HOU', 'DEN', 'WAS', 'CHI', 'TB', 'SF']

dpassing_plays = pbp_data_2023[pbp_data_2023['play_type'] == 'pass']

dpassing_plays_filtered = dpassing_plays[dpassing_plays['defteam'].isin(my_teams)]

dpassing_yards_per_game = dpassing_plays_filtered.groupby(['defteam', 'game_id'])['passing_yards'].sum().reset_index()

games_per_team = dpassing_yards_per_game.groupby('defteam')['game_id'].nunique().reset_index()
games_per_team.columns = ['team', 'total_games']

total_dpassing_yards_per_team = dpassing_yards_per_game.groupby('defteam')['passing_yards'].sum().reset_index()
total_dpassing_yards_per_team.columns = ['team', 'total_dpassing_yards']

team_stats = pd.merge(total_dpassing_yards_per_team, games_per_team, on='team')

team_stats['avg_dpassing_yards_per_game'] = team_stats['total_dpassing_yards'] / team_stats['total_games']

team_stats = team_stats.sort_values(by='avg_dpassing_yards_per_game', ascending=False)

print(team_stats[['team', 'total_dpassing_yards', 'total_games', 'avg_dpassing_yards_per_game']])

2023 done.
Downcasting floats.
  team  total_dpassing_yards  total_games  avg_dpassing_yards_per_game
7  WAS                4627.0           17                   272.176471
6   TB                5105.0           19                   268.684211
1  DEN                4302.0           17                   253.058824
2  HOU                4793.0           19                   252.263158
0  CHI                4245.0           17                   249.705882
3  MIA                4394.0           18                   244.111111
4  PIT                4367.0           18                   242.611111
5   SF                4749.0           20                   237.450000


## 2023 Rushing Defense

In [ ]:
pbp_data_2023 = nfl.import_pbp_data([2023])

my_teams = ['MIA', 'PIT', 'HOU', 'DEN', 'WAS', 'CHI', 'TB', 'SF']

drush_plays = pbp_data_2023[pbp_data_2023['play_type'] == 'run']

drush_plays_filtered = drush_plays[drush_plays['defteam'].isin(my_teams)]

drush_yards_per_game = drush_plays_filtered.groupby(['defteam', 'game_id'])['rushing_yards'].sum().reset_index()

games_per_team = drush_yards_per_game.groupby('defteam')['game_id'].nunique().reset_index()
games_per_team.columns = ['team', 'total_games']

total_drush_yards_per_team = drush_yards_per_game.groupby('defteam')['rushing_yards'].sum().reset_index()
total_drush_yards_per_team.columns = ['team', 'total_drush_yards']

team_stats = pd.merge(total_drush_yards_per_team, games_per_team, on='team')

team_stats['avg_drush_yards_per_game'] = team_stats['total_drush_yards'] / team_stats['total_games']

team_stats = team_stats.sort_values(by='avg_drush_yards_per_game', ascending=False)

print(team_stats[['team', 'total_drush_yards', 'total_games', 'avg_drush_yards_per_game']])

2023 done.
Downcasting floats.
  team  total_drush_yards  total_games  avg_drush_yards_per_game
1  DEN             2344.0           17                137.882353
7  WAS             2166.0           17                127.411765
4  PIT             2147.0           18                119.277778
2  HOU             1935.0           19                101.842105
3  MIA             1817.0           18                100.944444
5   SF             1982.0           20                 99.100000
6   TB             1780.0           19                 93.684211
0  CHI             1482.0           17                 87.176471


##**Major Takeaways**

- The Houston Texans far and away showed the greatest improvement between the 2022 and '23 seasons, increasing their win total from 3 wins to 11 wins and reaching the NFL playoffs, even winning a game in the Wild Card round. Much of this dramatic improvement can be attributed to the addition of rookie QB C.J. Stroud. In 2022, with Davis Mills at QB, the Texans struggled to the tune of 214 Passing yards per game. In 2023 with Stroud, that number was 265 passing yards per game. Beyond this, Mills' "bad throw" rate was a whopping 23.2% with a brutal "on-target" % of 70%, well below the other QB's I analyzed in this list. Stroud proved to be a large improvement in both categories, posting numbers of 20% and 75%.

- The numbers I pulled supported the infamous reputation of the Chicago Bears to be a run-first, poor-passing offense. In both 2022 and 2023, the Bears led my list of teams in rushing yards per game and were last in passing yards per game. Amazingly, the Bears averaged only 152 yards passing per game in 2022! While former QB Justin Fields is a dynamic athlete with an ability to make breathtaking plays, his passing ability simply never developed in Chicago to a place where they could reliably and effectively use his arm. That's why they drafted Caleb Williams with the no.1 overall pick last April.

- The consensus top team in my list of selected franchises for this project, the 49ers success appears to have a lot to do with a dominant defense. In both 2022 and 2023, the 49ers reached the postseason, including a super bowl appearance last season. In both 2022 and 2023, the 49ers defense allowed less than 100 yards per game, while improving their pass defense greatly between '22-'23. The offense more than held their own as well, ranking #2 among my teams in both passing and rushing offense in 2023. Interestingly enough, the 49ers have maintained and built upon their offensive success while changing QB's, with Jimmy Garoppolo and Trey Lance both seing extensive reps before coach Mike Shanahan settled on Brock Purdy as the face of the franchise. Those are the ingredients of a well-run franchise that hopes to continue competing for super bowls in the future.



##**Alignment with Research**
- In the previous research assignment, I asserted that two of my franchises (the Miami Dolphins and the San Francisco 49ers) in particular are known for creative, balanced, and explosive offenses. I think that assertion is supported by the research in this assignment:
  - Miami Dolphins offensive rankings (among my teams)
    - Passing: #1
    - Rushing: #3
  - San Francisco 49ers offensive rankings (among my teams)
    - Passing: #2
    - Rushing: #2

- In addition, I believe this research aligns with the common thought that QB play is the strongest indicator of a team's success. I have previously pointed out with the Houston Texans struggles in 2022 with Davis Mills at QB versus their success in 2023 with rising star C.J. Stroud at the helm, but this isn't the only example in my 8-team scope. In the opposite direction, the Washington Commanders win total decreased 4 games from somewhat-competitive 8-9 record in 2022, to an abysmal 4-13 record in 2023. This correlates with the Commanders' change from the servicable Taylor Heinicke to a young, inexperienced QB in Sam Howell.